# Mais Ouvidos

In [0]:
# Importando
from pyspark.sql.functions import col, round, year, month, concat, lit, lpad
from pyspark.sql import DataFrame
from functools import reduce

In [0]:
# Lendo todos arquivos na pasta e guardando em uma lista
folder = dbutils.fs.ls("dbfs:/FileStore/SpotifyData/StreamHistory/")

dataframes = []

for file in folder:
  json_file = spark.read.option("multiline", "true").option("mode", "PERMISSIVE").json(file[0])
  dataframes.append(json_file)

In [0]:
# Juntando arquivos lidos da lista
streamHistory = reduce(DataFrame.unionAll, dataframes)
streamHistory.count()

Out[11]: 35762

In [0]:
# Verificando colunas
print(streamHistory.columns)
# Verificando tipos
print(streamHistory.dtypes)

['artistName', 'endTime', 'msPlayed', 'trackName']
[('artistName', 'string'), ('endTime', 'string'), ('msPlayed', 'bigint'), ('trackName', 'string')]

In [0]:
# Renomeando colunas
streamHistory = streamHistory\
  .withColumnRenamed('msPlayed', 'minutesPlayed')\
  .withColumnRenamed('endTime', 'date')

In [0]:
# Tranformando milissegundos em minutos e deixando com duas casas decimais
streamHistory = streamHistory.withColumn("minutesPlayed", round((col("minutesPlayed")/60000), 2))

In [0]:
# Verificando
display(streamHistory.orderBy(col('date').desc()).take(20))

artistName,date,minutesPlayed,trackName
Gryffin,2021-05-09 16:55,3.69,Remember (with ZOHARA)
Au/Ra,2021-05-09 16:51,3.35,Ideas
AJR,2021-05-09 16:48,2.85,Bang!
The Unlikely Candidates,2021-05-09 16:45,3.07,High Low
Gryffin,2021-05-09 16:42,3.64,Cry
Tony Igy,2021-05-09 16:38,2.61,Astronomia - Never Go Home
salem ilese,2021-05-09 16:35,2.66,Roses to His Ex
Alan Walker,2021-05-09 16:33,2.8,Fake A Smile
Zeds Dead,2021-05-09 16:30,3.58,Frontlines
Gryffin,2021-05-09 16:26,3.97,All You Need To Know (feat. Calle Lehmann)


In [0]:
# Filtrando dados para análise
streamHistory = streamHistory.filter((col("trackName") != "Unknown Track") | (col("artistName") != "Unknown Artist"))
streamHistory = streamHistory.filter(col("minutesPlayed") != 0)

In [0]:
#Verificando
display(streamHistory.orderBy(col('date').desc()).take(20))
streamHistory.count()

artistName,date,minutesPlayed,trackName
Gryffin,2021-05-09 16:55,3.69,Remember (with ZOHARA)
Au/Ra,2021-05-09 16:51,3.35,Ideas
AJR,2021-05-09 16:48,2.85,Bang!
The Unlikely Candidates,2021-05-09 16:45,3.07,High Low
Gryffin,2021-05-09 16:42,3.64,Cry
Tony Igy,2021-05-09 16:38,2.61,Astronomia - Never Go Home
salem ilese,2021-05-09 16:35,2.66,Roses to His Ex
Alan Walker,2021-05-09 16:33,2.8,Fake A Smile
Zeds Dead,2021-05-09 16:30,3.58,Frontlines
Gryffin,2021-05-09 16:26,3.97,All You Need To Know (feat. Calle Lehmann)


Out[17]: 34985

## Separando por anos

In [0]:
# Criando coluna de ano a partir da data
streamHistory_year = streamHistory.withColumn("year", year(col("date")))

In [0]:
# Verificando
display(streamHistory_year.orderBy(col('date').desc()).take(20))

artistName,date,minutesPlayed,trackName,year
Gryffin,2021-05-09 16:55,3.69,Remember (with ZOHARA),2021
Au/Ra,2021-05-09 16:51,3.35,Ideas,2021
AJR,2021-05-09 16:48,2.85,Bang!,2021
The Unlikely Candidates,2021-05-09 16:45,3.07,High Low,2021
Gryffin,2021-05-09 16:42,3.64,Cry,2021
Tony Igy,2021-05-09 16:38,2.61,Astronomia - Never Go Home,2021
salem ilese,2021-05-09 16:35,2.66,Roses to His Ex,2021
Alan Walker,2021-05-09 16:33,2.8,Fake A Smile,2021
Zeds Dead,2021-05-09 16:30,3.58,Frontlines,2021
Gryffin,2021-05-09 16:26,3.97,All You Need To Know (feat. Calle Lehmann),2021


### Músicas

In [0]:
# Agrupando minutos tocados de cada música por ano
tracks_year = streamHistory_year\
  .groupby(['trackName', 'artistName', 'year'])\
  .sum('minutesPlayed')\
  .withColumnRenamed("sum(minutesPlayed)", "minutesPlayed")\
  .orderBy(col("minutesPlayed").desc())

# Arredondando soma de minutos tocados e deixando com duas casas decimais
tracks_year = tracks_year.withColumn("minutesPlayed", round(col("minutesPlayed"), 2))

In [0]:
# Verificando
display(tracks_year.take(50))

trackName,artistName,year,minutesPlayed
Rewind,Krewella,2020,869.86
Goddess,Krewella,2020,726.08
zer0,Krewella,2020,625.19
Anxiety,Krewella,2020,619.2
Scissors,Krewella,2020,600.14
Paradise,Krewella,2020,557.57
Like We,Krewella,2020,522.35
Mana - Extended Mix,Krewella,2020,494.92
Ghost,Krewella,2020,484.22
Good on You,Krewella,2020,418.84


In [0]:
# Verificando que o agrupamento ocorreu com Música/Artista e não só pelo nome da música
display(tracks_year.where(col("trackName") == "Bad Liar"))

trackName,artistName,year,minutesPlayed
Bad Liar,Imagine Dragons,2020,42.87
Bad Liar,Imagine Dragons,2021,40.52
Bad Liar,Krewella,2020,26.68
Bad Liar,Krewella,2021,15.41
Bad Liar,Selena Gomez,2020,0.03


In [0]:
# Filtrando Top 10 músicas de 2020
tracks2020Top10 = tracks_year.where(col("year") == "2020").take(10)
display(tracks2020Top10)

trackName,artistName,year,minutesPlayed
Rewind,Krewella,2020,869.86
Goddess,Krewella,2020,726.08
zer0,Krewella,2020,625.19
Anxiety,Krewella,2020,619.2
Scissors,Krewella,2020,600.14
Paradise,Krewella,2020,557.57
Like We,Krewella,2020,522.35
Mana - Extended Mix,Krewella,2020,494.92
Ghost,Krewella,2020,484.22
Good on You,Krewella,2020,418.84


In [0]:
# Filtrando Top 10 músicas de 2021
tracks2021Top10 = tracks_year.where(col("year") == "2021").take(10)
display(tracks2021Top10)

trackName,artistName,year,minutesPlayed
Cutthroat,Imagine Dragons,2021,384.99
Follow You,Imagine Dragons,2021,338.49
Christmas in June,AJR,2021,221.31
Hearts on Fire,ILLENIUM,2021,209.8
Fake A Smile,Alan Walker,2021,204.57
Way Less Sad,AJR,2021,176.03
3 O'Clock Things,AJR,2021,160.09
Ain't It Fun,Paramore,2021,146.31
OK Overture,AJR,2021,144.58
Space Melody (Edward Artemyev) (feat. Leony),VIZE,2021,144.31


### Artistas

In [0]:
# Agrupando minutos tocados de cada artista por ano
artists_year = streamHistory_year\
  .groupby(['artistName','year'])\
  .sum('minutesPlayed')\
  .withColumnRenamed("sum(minutesPlayed)", "minutesPlayed")\
  .orderBy(col("minutesPlayed").desc())

# Arredondando soma de minutos tocados e deixando com duas casas decimais
artists_year = artists_year.withColumn("minutesPlayed", round(col("minutesPlayed"), 2))

In [0]:
# Verificando
display(artists_year.take(20))

artistName,year,minutesPlayed
Krewella,2020,9296.82
ILLENIUM,2020,6114.84
Paramore,2020,3865.08
Paramore,2021,2862.11
AJR,2021,2398.2
Imagine Dragons,2021,2322.01
Imagine Dragons,2020,1751.77
I DONT KNOW HOW BUT THEY FOUND ME,2020,1707.53
ILLENIUM,2021,1562.09
Panic! At The Disco,2020,1457.46


In [0]:
# Filtrando Top 10 artistas de 2020
artists2020Top10 = artists_year.where(col("year") == "2020").take(10)
display(artists2020Top10)

artistName,year,minutesPlayed
Krewella,2020,9296.82
ILLENIUM,2020,6114.84
Paramore,2020,3865.08
Imagine Dragons,2020,1751.77
I DONT KNOW HOW BUT THEY FOUND ME,2020,1707.53
Panic! At The Disco,2020,1457.46
Halsey,2020,1368.8
Twenty One Pilots,2020,993.15
Little Mix,2020,823.52
R3HAB,2020,814.94


In [0]:
# Filtrando Top 10 artistas de 2021
artists2021Top10 = artists_year.where(col("year") == "2021").take(10)
display(artists2021Top10)

artistName,year,minutesPlayed
Paramore,2021,2862.11
AJR,2021,2398.2
Imagine Dragons,2021,2322.01
ILLENIUM,2021,1562.09
Krewella,2021,1199.93
Twenty One Pilots,2021,964.25
Gryffin,2021,896.79
The Tech Thieves,2021,886.97
Fall Out Boy,2021,689.81
Halsey,2021,641.93


## Separando por ano-mês

In [0]:
# Criando coluna de "ano-mes" a partit da data
streamHistory_year_month = streamHistory.withColumn("year-month",concat(year(col("date")), lit("-"), lpad(month(col("date")), 2, '0')))
display(streamHistory_year_month.orderBy(col('date').desc()).take(20))

artistName,date,minutesPlayed,trackName,year-month
Gryffin,2021-05-09 16:55,3.69,Remember (with ZOHARA),2021-05
Au/Ra,2021-05-09 16:51,3.35,Ideas,2021-05
AJR,2021-05-09 16:48,2.85,Bang!,2021-05
The Unlikely Candidates,2021-05-09 16:45,3.07,High Low,2021-05
Gryffin,2021-05-09 16:42,3.64,Cry,2021-05
Tony Igy,2021-05-09 16:38,2.61,Astronomia - Never Go Home,2021-05
salem ilese,2021-05-09 16:35,2.66,Roses to His Ex,2021-05
Alan Walker,2021-05-09 16:33,2.8,Fake A Smile,2021-05
Zeds Dead,2021-05-09 16:30,3.58,Frontlines,2021-05
Gryffin,2021-05-09 16:26,3.97,All You Need To Know (feat. Calle Lehmann),2021-05


### Música mais tocada por ano-mês

In [0]:
topMusicPerMonth = streamHistory_year_month.groupby(['trackName','artistName','year-month'])\
  .sum('minutesPlayed')\
  .withColumnRenamed("sum(minutesPlayed)", "minutesPlayed")\
  .orderBy(col("minutesPlayed").desc())

# Arredondando soma de minutos tocados e deixando com duas casas decimais
topMusicPerMonth = topMusicPerMonth.withColumn("minutesPlayed", round(col("minutesPlayed"), 2))

# Agrupando por minutos máximos por mês
maxMinutesMusic = topMusicPerMonth.groupBy('year-month').max('minutesPlayed').withColumnRenamed("max(minutesPlayed)", "minutesPlayed")

# Juntando para ver o nome da música com max minutos
topMusicPerMonth = topMusicPerMonth.join(maxMinutesMusic,on='minutesPlayed',how='leftsemi').orderBy(col('year-month'))
display(topMusicPerMonth)

minutesPlayed,trackName,artistName,year-month
105.74,Goddess,Krewella,2020-05
82.27,Spirals,ILLENIUM,2020-06
121.77,Rewind,Krewella,2020-07
91.38,Rewind,Krewella,2020-08
151.94,Nightlight,ILLENIUM,2020-09
67.29,Rewind,Krewella,2020-10
503.12,zer0,Krewella,2020-11
70.08,Sideways,Tisoki,2020-12
82.83,Ain't It Fun,Paramore,2021-01
69.06,Fake A Smile,Alan Walker,2021-02


### Artistas

In [0]:
topArtistPerMonth = streamHistory_year_month.groupby(['artistName','year-month'])\
  .sum('minutesPlayed')\
  .withColumnRenamed("sum(minutesPlayed)", "minutesPlayed")\
  .orderBy(col("minutesPlayed").desc())

# Arredondando soma de minutos tocados e deixando com duas casas decimais
topArtistPerMonth = topArtistPerMonth.withColumn("minutesPlayed", round(col("minutesPlayed"), 2))

# Agrupando por minutos máximos por mês
maxMinutesArtist = topArtistPerMonth.groupBy('year-month').max('minutesPlayed').withColumnRenamed("max(minutesPlayed)", "minutesPlayed")

# Juntando para ver o nome da música com max minutos
topArtistPerMonth = topArtistPerMonth.join(maxMinutesArtist,on='minutesPlayed',how='leftsemi').orderBy(col('year-month'))
display(topArtistPerMonth)

minutesPlayed,artistName,year-month
378.69,Krewella,2020-05
524.68,ILLENIUM,2020-06
951.89,Paramore,2020-07
1431.01,Paramore,2020-08
624.75,Paramore,2020-09
646.23,Krewella,2020-10
6327.3,Krewella,2020-11
1062.41,Krewella,2020-12
1582.81,Paramore,2021-01
767.04,Paramore,2021-02
